In [1]:
import pandas as pd
import numpy as np

#读取数据
train_price = pd.read_csv("ruc_Class25Q2_train_price.csv", low_memory=False)
test_price  = pd.read_csv("ruc_Class25Q2_test_price.csv",  low_memory=False)

#查看数据
print("=====Train Price Data Info=====")
print(train_price.shape)
print(train_price.columns)
print(train_price.head(3))
print(train_price.info())

print("\n=====Test Price Data Info=====")
print(test_price.shape)
print(test_price.columns)
print(test_price.info())

# 检查缺失值
print("\n=====Missing Values in Train Price=====")
print(train_price.isnull().sum().sort_values(ascending=False).head(20))

=====Train Price Data Info=====
(103871, 55)
Index(['城市', '区域', '板块', '环线', 'Price', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向',
       '建筑结构', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途',
       '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon',
       'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数',
       '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话',
       '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x',
       'coord_y', '客户反馈'],
      dtype='object')
   城市     区域     板块    环线         Price      房屋户型       所在楼层     建筑面积  \
0   0  109.0  150.0  二至三环  6.194049e+06  2室1厅1厨1卫  中楼层 (共5层)    52.3㎡   
1   0   65.0  299.0  五至六环  4.354153e+06  3室1厅1厨1卫   顶层 (共6层)  127.44㎡   
2   0   62.0  911.0  五至六环  3.321992e+06  3室2厅1厨2卫  低楼层 (共6层)  118.02㎡   

      套内面积 房屋朝向  ...     供水        供暖     供电       燃气费    供热费     停车位 停车费用  \
0      NaN  南 北  ...     民水      集中供暖     民电  2.61元/m³  30元/㎡   300.0   暂无  

In [2]:
#处理数据

#户型
import re
def house_features(house_type):
    #空值直接填0室0厅0卫
    if pd.isna(house_type) or house_type.strip() == '':
        return pd.Series({'室': 0, '厅': 0, '卫': 0})
    
    text = str(house_type)
    
    #提取“室”“厅”“卫”的数字
    room = re.search(r'(\d+)\s*室', str(house_type))
    hall = re.search(r'(\d+)\s*厅', str(house_type))
    bath = re.search(r'(\d+)\s*卫', str(house_type))
    room_alt = re.search(r'(\d+)\s*房间', str(house_type))
    
    #提取数字或设默认值
    rooms = int(room.group(1)) if room else (int(room_alt.group(1)) if room_alt else np.nan)
    halls = int(hall.group(1)) if hall else 0
    baths = int(bath.group(1)) if bath else np.nan
    
    return pd.Series({'室': rooms, '厅': halls, '卫': baths})

for df in [train_price, test_price]:
    df[['室', '厅', '卫']] = df['房屋户型'].apply(house_features)

#检查结果
print(train_price[['室', '厅', '卫']].head())
print(test_price[['室', '厅', '卫']].head())

   室  厅  卫
0  2  1  1
1  3  1  1
2  3  2  2
3  6  3  3
4  1  0  1
   室  厅  卫
0  3  2  2
1  2  1  1
2  3  1  2
3  2  1  1
4  3  2  2


In [3]:
#房屋用途
main_types = ['普通住宅', '商住两用', '别墅', '商业办公类']

def process_usage(df):
    df['房屋用途_处理后'] = df['房屋用途'].apply(lambda x: x if x in main_types else '其他')
    
    #生成哑变量（drop_first=True 避免多重共线性）
    df = pd.get_dummies(df, columns=['房屋用途_处理后'], prefix='用途', drop_first=True)
    return df

train_price = process_usage(train_price)
test_price = process_usage(test_price)

#检查结果
print(train_price.filter(like='用途_').head())

   用途_别墅  用途_商业办公类  用途_商住两用  用途_普通住宅
0  False     False    False     True
1  False     False    False     True
2  False     False    False     True
3   True     False    False    False
4  False      True    False    False


In [4]:
#所在楼层

def process_floor_vectorized(df, villa_col='用途_别墅', region_col='板块', floor_col='所在楼层'):
    """
    处理楼层列，生成三列：
    rel_floor: 相对高度（0~1，底层0.2，顶层0.9），地下室为0
    total_floor: 总楼层数
    est_floor: 估计的该房源楼层数（绝对层数），地下室为-1
    别墅直接填为0
    """
    rel_map = {'地下室': 0.0, '底': 0.1, '低':0.3, '中':0.5, '高':0.7, '顶':0.9}
    
    #提取总楼层数
    total_floors = df[floor_col].str.extract(r'共(\d+)层')[0].astype(float)
    
    #提取相对楼层
    def map_rel(text):
        for k,v in rel_map.items():
            if k in str(text):
                return v
        return 0.5  # 默认中层
    rel_floor = df[floor_col].map(map_rel)
    
    #计算绝对楼层
    est_floor = np.round(rel_floor * total_floors).fillna(1).astype(float)

    df['rel_floor'] = rel_floor
    df['total_floor'] = total_floors
    df['est_floor'] = est_floor
    
    #地下室绝对楼层为-1
    est_floor[df[floor_col].str.contains('地下室', na=False)] = -1
    
    #别墅全部填为0
    villa_idx = df[df[villa_col]==1].index
    df.loc[villa_idx, ['rel_floor','est_floor','total_floor']] = 0
    
    return df

# 应用
train_price = process_floor_vectorized(train_price)
test_price = process_floor_vectorized(test_price)

# 查看结果
print(train_price[['所在楼层','rel_floor','total_floor','est_floor','用途_别墅']].head(10))

         所在楼层  rel_floor  total_floor  est_floor  用途_别墅
0   中楼层 (共5层)        0.5          5.0        2.0  False
1    顶层 (共6层)        0.9          6.0        5.0  False
2   低楼层 (共6层)        0.3          6.0        2.0  False
3    底层 (共2层)        0.0          0.0        0.0   True
4  中楼层 (共10层)        0.5         10.0        5.0  False
5   地下室 (共2层)        0.0          2.0        0.0  False
6  高楼层 (共20层)        0.7         20.0       14.0  False
7   地下室 (共0层)        0.0          0.0        0.0  False
8  高楼层 (共22层)        0.7         22.0       15.0  False
9    底层 (共3层)        0.1          3.0        0.0  False


In [5]:
#建筑面积、套内面积

def process_area(df, area_col='建筑面积', inner_col='套内面积', group_cols=['板块','房屋户型']):
    # 转为数值型
    def str_to_float(x):
        if pd.isnull(x):
            return np.nan
        return float(str(x).replace('㎡','').strip())
    
    df[area_col] = df[area_col].apply(str_to_float)
    df[inner_col] = df[inner_col].apply(str_to_float)
    
    # 用同板块+户型中位数填充套内面积缺失
    if all(col in df.columns for col in group_cols):
        df[inner_col] = df.groupby(group_cols)[inner_col].transform(lambda x: x.fillna(x.median()))
    
    # 剩余缺失用全局中位数填充
    df[inner_col] = df[inner_col].fillna(df[inner_col].median())
    
    # 计算得房率
    df['得房率'] = df[inner_col] / df[area_col]
    
    return df

train_price = process_area(train_price)
test_price = process_area(test_price)

#检查结果
print(train_price[['建筑面积','套内面积','得房率']].head())
print(test_price[['建筑面积','套内面积','得房率']].head())

     建筑面积     套内面积       得房率
0   52.30   38.575  0.737572
1  127.44  123.700  0.970653
2  118.02  101.950  0.863837
3  293.23  293.230  1.000000
4   39.85   29.940  0.751317
     建筑面积    套内面积       得房率
0  282.02   74.43  0.263917
1   88.42   71.78  0.811807
2  175.52  139.86  0.796832
3  106.13   71.59  0.674550
4  116.80   74.43  0.637243


In [6]:
#房屋朝向
directions = ['东', '南', '西', '北', '东南', '东北', '西南', '西北']

def process_orientation(df, col='房屋朝向'):
    for dir_ in directions:
        #有这个方向标1，否则0
        df[f'朝向_{dir_}'] = df[col].apply(lambda x: 1 if pd.notna(x) and dir_ in x else 0)
    return df

train_price = process_orientation(train_price)
test_price = process_orientation(test_price)

#检查结果
print(train_price[[f'朝向_{d}' for d in directions]].head())

   朝向_东  朝向_南  朝向_西  朝向_北  朝向_东南  朝向_东北  朝向_西南  朝向_西北
0     0     1     0     1      0      0      0      0
1     0     1     0     1      0      0      0      0
2     1     1     0     0      1      0      0      0
3     1     1     1     1      0      0      0      0
4     0     1     0     0      0      0      0      0


In [7]:
#建筑结构
main_structures = ['钢混结构', '混合结构', '砖混结构', '未知结构']

def process_structure(df, col='建筑结构'):
    # 先把缺失值填充为'未知结构'
    df[col] = df[col].fillna('未知结构')
    
    # 归类：不在main_structures中的都标为'其他'
    df[col] = df[col].apply(lambda x: x if x in main_structures else '其他')
    
    #生成哑变量
    dummies = pd.get_dummies(df[col], prefix='结构', drop_first=True)
    
    # 合并回原数据
    df = pd.concat([df, dummies], axis=1)
    return df

# 应用到训练集和测试集
train_price = process_structure(train_price)
test_price = process_structure(test_price)

# 检查结果
print(train_price[[f'结构_{s}' for s in main_structures]].head())

   结构_钢混结构  结构_混合结构  结构_砖混结构  结构_未知结构
0    False     True    False    False
1    False     True    False    False
2     True    False    False    False
3    False     True    False    False
4     True    False    False    False


In [8]:
#装修情况

#填充空值为'其他'
train_price['装修情况'] = train_price['装修情况'].fillna('其他')
test_price['装修情况'] = test_price['装修情况'].fillna('其他')

#生成哑变量
train_renov = pd.get_dummies(train_price['装修情况'], prefix='装修', drop_first=True)
test_renov = pd.get_dummies(test_price['装修情况'], prefix='装修', drop_first=True)

train_price = pd.concat([train_price, train_renov], axis=1)
test_price = pd.concat([test_price, test_renov], axis=1)

print(train_price.filter(like='装修_').head())

   装修_毛坯  装修_简装  装修_精装
0  False  False   True
1  False  False   True
2  False   True  False
3  False  False   True
4  False  False   True


In [9]:
#梯户比、电梯

# 中文数字到阿拉伯数字映射
cn_num = {
    '一': 1, '二': 2, '两':2,'三': 3, '四': 4, '五': 5, 
    '六': 6, '七': 7, '八': 8, '九': 9, '十': 10,
    '十一': 11, '十二': 12, '十三': 13, '十四': 14, '十五': 15
}

def chinese_to_int(text):
    if text in cn_num:
        return cn_num[text]
    #处理“十一”“十二”这类组合
    elif '十' in text:
        parts = text.split('十')
        if parts[0] == '':
            return 10 + cn_num.get(parts[1], 0)
        else:
            return cn_num.get(parts[0], 0)*10 + cn_num.get(parts[1],0)
    return 0

def compute_lift_ratio(text):
    if pd.isna(text):
        return 0
    match = re.match(r'(\D+)梯(\D+)户', str(text))
    if match:
        lifts = chinese_to_int(match.group(1))
        households = chinese_to_int(match.group(2))
        return lifts / households if households != 0 else 0
    return 0

#应用
train_price['梯户比'] = train_price['梯户比例'].apply(compute_lift_ratio)
test_price['梯户比'] = test_price['梯户比例'].apply(compute_lift_ratio)

#检查
print(train_price[['梯户比例','梯户比']].head())

    梯户比例       梯户比
0   一梯三户  0.333333
1   一梯两户  0.500000
2   一梯五户  0.200000
3    NaN  0.000000
4  两梯十一户  0.181818


In [10]:
#配备电梯
def process_elevator(df, col='配备电梯'):
    #空值填“无”
    df[col] = df[col].fillna('无')
    
    #映射为哑变量：有电梯=1，无电梯=0
    df['电梯'] = df[col].apply(lambda x: 1 if '有' in str(x) else 0)
    
    return df

train_price = process_elevator(train_price)
test_price = process_elevator(test_price)

# 检查结果
print(train_price[['配备电梯','电梯']].head(10))

  配备电梯  电梯
0    无   0
1    无   0
2    有   1
3    无   0
4    有   1
5    无   0
6    有   1
7    无   0
8    有   1
9    无   0


In [11]:
#交易时间、上次交易
def process_trade_dates(df):
    # 统一时间格式
    df['交易时间'] = pd.to_datetime(df['交易时间'], errors='coerce')
    df['上次交易'] = pd.to_datetime(df['上次交易'], errors='coerce')
    df['交易年份'] = df['交易时间'].dt.year

    # 新增首次交易标记
    df['首次交易'] = df['上次交易'].isna().astype(int)

    # 计算交易间隔天数
    df['交易间隔天数'] = (df['交易时间'] - df['上次交易']).dt.days

    # 首次交易的交易间隔填0
    df.loc[df['首次交易'] == 1, '交易间隔天数'] = 0

    return df

train_price = process_trade_dates(train_price)
test_price = process_trade_dates(test_price)

#检查结果
print(train_price[['交易年份','交易间隔天数','首次交易']].head())
print(test_price[['交易年份','交易间隔天数','首次交易']].head())

   交易年份  交易间隔天数  首次交易
0  2021  2798.0     0
1  2020  3611.0     0
2  2020  2584.0     0
3  2023  2158.0     0
4  2019  1017.0     0
   交易年份  交易间隔天数  首次交易
0  2025  6229.0     0
1  2025  5144.0     0
2  2025  5445.0     0
3  2025  6044.0     0
4  2025  9220.0     0


In [12]:
#房屋年限
def process_house_age(df, col='房屋年限'):
    # 空值填充为“未满两年”
    df[col] = df[col].fillna('未满两年')
    
    #定义要保留的三类
    age_categories = ['满五年', '满两年', '未满两年']

    #创建哑变量
    df_age_dummies = pd.get_dummies(df[col], prefix='房屋年限', drop_first=True)
    
    # 合并到原表
    df = pd.concat([df, df_age_dummies], axis=1)
    
    return df

# 应用
train_price = process_house_age(train_price)
test_price = process_house_age(test_price)

# 检查
print(train_price[['房屋年限'] + [c for c in train_price.columns if '房屋年限_' in c]].head(10))

  房屋年限  房屋年限_满两年  房屋年限_满五年
0  满五年     False      True
1  满五年     False      True
2  满五年     False      True
3  满五年     False      True
4  满五年     False      True
5  满五年     False      True
6  满五年     False      True
7  满五年     False      True
8  满五年     False      True
9  满五年     False      True


In [13]:
#产权所属
def process_property_right(df, col='产权所属'):
    # 将“共有”标记为1，其余为0
    df['产权_共有'] = df[col].apply(lambda x: 1 if x == '共有' else 0)
    return df

# 应用
train_price = process_property_right(train_price)
test_price = process_property_right(test_price)

# 检查结果
print(train_price[['产权所属', '产权_共有']].head(10))

  产权所属  产权_共有
0  非共有      0
1  非共有      0
2  非共有      0
3  非共有      0
4  非共有      0
5  非共有      0
6  非共有      0
7  非共有      0
8  非共有      0
9   共有      1


In [14]:
#周边配套
def process_surroundings(df, col='周边配套'):
    df['周边_医院'] = df[col].apply(lambda x: 1 if isinstance(x, str) and '医院' in x else 0)
    df['周边_公园'] = df[col].apply(lambda x: 1 if isinstance(x, str) and '公园' in x else 0)
    # 商业类包括 超市、购物、商场、商业
    df['周边_商业'] = df[col].apply(lambda x: 1 if isinstance(x, str) and any(k in x for k in ['超市','购物','商场','商业']) else 0)
    return df

# 应用
train_price = process_surroundings(train_price)
test_price = process_surroundings(test_price)

# 检查结果
print(train_price[['周边_医院','周边_公园','周边_商业']].head(10))

   周边_医院  周边_公园  周边_商业
0      1      1      1
1      1      1      0
2      0      0      0
3      1      0      0
4      0      0      0
5      0      0      0
6      0      0      1
7      0      0      1
8      0      0      0
9      0      0      1


In [15]:
#交通出行
def process_transport(df, col='交通出行'):
    df['交通_地铁'] = df[col].apply(lambda x: 1 if isinstance(x, str) and '地铁' in x else 0)
    df['交通_公交'] = df[col].apply(lambda x: 1 if isinstance(x, str) and '公交' in x else 0)
    return df

# 应用
train_price = process_transport(train_price)
test_price = process_transport(test_price)

# 检查结果
print(train_price[['交通_地铁','交通_公交']].head(10))

   交通_地铁  交通_公交
0      0      0
1      0      0
2      0      0
3      1      1
4      1      0
5      0      0
6      1      1
7      1      1
8      0      1
9      0      1


In [16]:
#建筑年代
def process_building_year(df, col='建筑年代', region_col='板块'):
    """
    格式如 '1955-2000年' 或 '2000年'
    提取平均年份
    缺失值用所在板块的中位数填充
    """
    def extract_avg_year(text):
        if pd.isna(text):
            return np.nan
        # 提取所有4位数字年份
        years = re.findall(r'\d{4}', str(text))
        if len(years) == 2:
            return (float(years[0]) + float(years[1])) / 2
        elif len(years) == 1:
            return float(years[0])
        else:
            return np.nan

    # 转换为数值列
    df['平均建筑年代'] = df[col].apply(extract_avg_year)

    # 按板块计算中位数（这里用 numeric 列，而不是原始 object 列）
    median_by_region = df.groupby(region_col)['平均建筑年代'].transform('median')
    df['平均建筑年代'] = df['平均建筑年代'].fillna(median_by_region)

    # 若板块中也都是空，再用全局中位数
    df['平均建筑年代'] = df['平均建筑年代'].fillna(df['平均建筑年代'].median())

    return df

# 应用
train_price = process_building_year(train_price)
test_price = process_building_year(test_price)

print(train_price[['建筑年代', '板块', '平均建筑年代']].head(10))

         建筑年代      板块  平均建筑年代
0  1955-2000年   150.0  1977.5
1       2005年   299.0  2005.0
2  2011-2012年   911.0  2011.5
3  2009-2011年  1102.0  2010.0
4  2003-2018年   295.0  2010.5
5  2007-2010年    78.0  2008.5
6  2011-2014年  1126.0  2012.5
7  2011-2014年  1126.0  2012.5
8  1997-1998年   878.0  1997.5
9  2004-2006年   547.0  2005.0


In [17]:
#房屋总数
def process_total_households(df, col='房屋总数', group_col='区域'):
    #去掉单位并提取数字
    df[col] = (
        df[col]
        .astype(str)
        .str.replace('户', '', regex=False)
        .str.extract(r'(\d+)')
        .astype(float)
    )

    #按区域中位数填补空值
    median_by_region = df.groupby(group_col)[col].transform('median')
    df[col] = df[col].fillna(median_by_region)

    #若区域中也都是空，再用全局中位数
    df[col] = df[col].fillna(df[col].median())

    return df

train_price = process_total_households(train_price, '房屋总数', '区域')
test_price = process_total_households(test_price, '房屋总数', '区域')

print(train_price[['区域', '房屋总数']].head(10))

      区域    房屋总数
0  109.0  1317.0
1   65.0  2317.0
2   62.0  1554.0
3  123.0    66.0
4   81.0  1685.0
5  123.0   430.0
6  112.0  1100.0
7  112.0  1100.0
8   81.0   522.0
9   28.0   396.0


In [18]:
#绿化率
def process_green_rate(df, col='绿 化 率'):
    df[col] = df[col].fillna('0%') 
    df[col] = df[col].str.replace('%','').astype(float)
    return df

# 应用
train_price = process_green_rate(train_price)
test_price = process_green_rate(test_price)

# 查看结果
print(train_price[['绿 化 率']].head(10))

   绿 化 率
0   30.0
1   30.0
2   30.0
3   40.1
4   60.0
5   40.0
6   35.0
7   35.0
8   25.0
9   40.0


In [19]:
def process_property_fee(df, col='物 业 费', region_col='区域'):
    """
    处理物业费列：
    格式如 '1-1.63元/月/㎡' 或空-提取平均值
    缺失值用所在区域的中位数填充，如果区域全空再用全局中位数
    """
    def extract_mid(s):
        if pd.isna(s):
            return np.nan
        # 提取数字
        nums = re.findall(r'[\d\.]+', str(s))
        nums = [float(x) for x in nums]
        if len(nums) == 1:
            return nums[0]
        elif len(nums) >= 2:
            return np.mean(nums[:2])
        else:
            return np.nan
    
    # 提取物业费数值
    df[col+'_num'] = df[col].map(extract_mid)

    # 按区域中位数填充
    median_by_region = df.groupby(region_col)[col+'_num'].transform('median')
    df[col+'_num'] = df[col+'_num'].fillna(median_by_region)

    # 区域中也全空的用全局中位数
    df[col+'_num'] = df[col+'_num'].fillna(df[col+'_num'].median())

    return df

# 应用
train_price = process_property_fee(train_price)
test_price = process_property_fee(test_price)

print(train_price[['物 业 费', '物 业 费_num']].head(10))

            物 业 费  物 业 费_num
0   1.3-1.65元/月/㎡      1.475
1       0.65元/月/㎡      0.650
2  1.98-2.98元/月/㎡      2.480
3     3-6.07元/月/㎡      4.535
4    4.8-5.5元/月/㎡      5.150
5          7元/月/㎡      7.000
6  2.67-2.68元/月/㎡      2.675
7  2.67-2.68元/月/㎡      2.675
8       2.75元/月/㎡      2.750
9    1.8-2.5元/月/㎡      2.150


In [20]:
#建筑结构_comm
def process_building_type(df, col='建筑结构_comm'):
    types = ['塔楼', '板楼', '塔板结合', '平房']
    
    # 填空值为 ''
    df[col] = df[col].fillna('')
    
    #遍历每个类型生成哑变量
    for t in types:
        df[f'结构_{t}'] = df[col].apply(lambda x: t in x)
    
    return df

train_price = process_building_type(train_price, col='建筑结构_comm')
test_price = process_building_type(test_price, col='建筑结构_comm')

print(train_price[['建筑结构_comm'] + [f'结构_{t}' for t in ['塔楼','板楼','塔板结合','平房']]].head(10))

    建筑结构_comm  结构_塔楼  结构_板楼  结构_塔板结合  结构_平房
0       板楼/平房  False   True    False   True
1          板楼  False   True    False  False
2          板楼  False   True    False  False
3          板楼  False   True    False  False
4  塔楼/板楼/塔板结合   True   True     True  False
5          板楼  False   True    False  False
6     板楼/塔板结合  False   True     True  False
7     板楼/塔板结合  False   True     True  False
8          塔楼   True  False    False  False
9          板楼  False   True    False  False


In [21]:
#产权描述
def process_property_type(df, col='产权描述'):
    """
    根据产权描述列生成三个哑变量：
    - 产权_高价：完全产权（商品房）
    - 产权_中价：部分限制（已购公房、央产房、共有产权房等）
    - 产权_低价：限制明显（经济适用房、私产、自建、小产权等）
    若某行无法识别（空白或其他），默认为中价档
    """
    df[col] = df[col].astype(str).fillna('')

    # 高价档：完全产权
    high_pattern = r'商品房'

    # 中价档：部分限制（已购公房、央产、共有产权）
    mid_pattern = r'已购公房|央产房|共有|使用权'

    # 低价档：经济适用、私产、自建、小产权
    low_pattern = r'经济适用|私产|自建|小产'

    # 生成哑变量（用str.contains匹配所有可能分隔形式）
    df['产权_高价'] = df[col].str.contains(high_pattern, regex=True)
    df['产权_中价'] = df[col].str.contains(mid_pattern, regex=True)
    df['产权_低价'] = df[col].str.contains(low_pattern, regex=True)

    # 若全部为False（无法识别或空白），默认为高价（完全产权）
    none_idx = ~(df[['产权_高价', '产权_中价', '产权_低价']].any(axis=1))
    df.loc[none_idx, '产权_中价'] = True

    # 转换为int类型
    df[['产权_高价', '产权_中价', '产权_低价']] = df[['产权_高价', '产权_中价', '产权_低价']].astype(int)

    return df

# 应用到训练集和测试集
train_price = process_property_type(train_price, '产权描述')
test_price = process_property_type(test_price, '产权描述')

# 查看结果
print(train_price[['产权描述','产权_高价','产权_中价','产权_低价']].head(10))

              产权描述  产权_高价  产权_中价  产权_低价
0  商品房/已购公房/央产房/私产      1      1      1
1   商品房/一类经济适用房/私产      1      0      1
2        商品房/限价商品房      1      0      0
3              商品房      1      0      0
4              商品房      1      0      0
5              商品房      1      0      0
6              商品房      1      0      0
7              商品房      1      0      0
8      商品房/已购公房/私产      1      1      1
9           商品房/私产      1      0      1


In [22]:
def process_supply_columns(df):
    # 供水供电（民水民电=1，其他=0）
    df['水电'] = df['供水'].fillna('').str.contains('民水', na=False).astype(int)

    # 供暖（集中供暖=1，其他=0）
    df['供暖'] = df['供暖'].fillna('').str.contains('集中供暖', na=False).astype(int)
    
    return df

# 应用
train_price = process_supply_columns(train_price)
test_price = process_supply_columns(test_price)

# 检查结果
print(train_price[['水电','供暖']].head(10))

   水电  供暖
0   1   1
1   1   0
2   1   1
3   1   0
4   1   1
5   1   0
6   1   1
7   1   1
8   1   1
9   1   1


In [23]:
#燃气费、供热费
def process_fee(df, col):
    def str_to_mean(x):
        if pd.isna(x):
            return 0.0
        #提取数字
        nums = re.findall(r'[\d.]+', str(x))
        if len(nums) == 0:
            return 0.0
        nums = [float(n) for n in nums]
        return np.mean(nums)

    df[col] = df[col].apply(str_to_mean)
    return df

train_price = process_fee(train_price, '燃气费')
train_price = process_fee(train_price, '供热费')
test_price = process_fee(test_price, '燃气费')
test_price = process_fee(test_price, '供热费')

print(train_price[['燃气费','供热费']].head(10))

    燃气费   供热费
0  2.61  30.0
1  2.61   0.0
2  2.61  30.0
3  2.62   0.0
4  2.62  37.5
5  2.61   0.0
6  2.61  30.0
7  2.61  30.0
8  2.61  30.0
9  2.61  30.0


In [24]:
#停车位
def process_parking_spots(df, col='停车位', region_col='区域'):
    # 转换为数值类型，非数值的设为 NaN
    df[col + '_处理后'] = pd.to_numeric(df[col], errors='coerce')

    # 按区域中位数填充缺失值
    median_by_region = df.groupby(region_col)[col + '_处理后'].transform('median')
    df[col + '_处理后'] = df[col + '_处理后'].fillna(median_by_region)

    # 区域中也全空的用全局中位数
    df[col + '_处理后'] = df[col + '_处理后'].fillna(df[col + '_处理后'].median())

    return df

train_price = process_parking_spots(train_price)
test_price = process_parking_spots(test_price)

print(train_price[['停车位', '停车位_处理后']].head(10))

# 容积率
def process_floor_area_ratio(df, col='容 积 率', region_col='区域'):
    # 按区域中位数填充缺失值
    median_by_region = df.groupby(region_col)[col].transform('median')
    df[col + '_处理后'] = df[col].fillna(median_by_region)

    # 区域中也全空的用全局中位数
    df[col + '_处理后'] = df[col + '_处理后'].fillna(df[col + '_处理后'].median())

    return df

train_price = process_floor_area_ratio(train_price)
test_price = process_floor_area_ratio(test_price)

print(train_price[['容 积 率', '容 积 率_处理后']].head(10))

      停车位  停车位_处理后
0   300.0    300.0
1  1550.0   1550.0
2   324.0    324.0
3   500.0    500.0
4  1800.0   1800.0
5   743.0    743.0
6  2289.0   2289.0
7  2289.0   2289.0
8   642.0    642.0
9   600.0    600.0
   容 积 率  容 积 率_处理后
0   3.00       3.00
1   1.73       1.73
2   1.70       1.70
3   1.00       1.00
4   1.58       1.58
5   1.20       1.20
6   1.50       1.50
7   1.50       1.50
8   2.80       2.80
9   0.60       0.60


In [25]:
#停车费用
def process_parking_fee(df, col='停车费用', region_col='区域'):
    """
    处理停车费用列：
    - '暂无' -> 0
    - '地上150元/月/位，地下500元/月/位' -> 提取第一个数字 (150)
    - 其他非数字描述 -> 0
    - 缺失值用所在区域中位数填充
    - 区域中全空的用全局中位数
    """
    def extract_fee(val):
        if pd.isna(val):
            return np.nan  # 先用 NaN
        val_str = str(val)
        if '暂无' in val_str:
            return np.nan
        # 尝试提取第一个数字
        match = re.search(r'(\d+\.?\d*)', val_str)
        if match:
            return float(match.group(1))
        return np.nan  # 非数字的先设为 NaN

    # 提取停车费用
    df['停车费用_处理后'] = df[col].apply(extract_fee)

    # 按区域中位数填充缺失值
    median_by_region = df.groupby(region_col)['停车费用_处理后'].transform('median')
    df['停车费用_处理后'] = df['停车费用_处理后'].fillna(median_by_region)

    # 区域中也全空的用全局中位数
    df['停车费用_处理后'] = df['停车费用_处理后'].fillna(df['停车费用_处理后'].median())

    return df

# 应用
train_price = process_parking_fee(train_price)
test_price = process_parking_fee(test_price)

# 检查结果
print(train_price[['停车费用', '停车费用_处理后']].head(10))

   停车费用  停车费用_处理后
0    暂无     150.0
1   150     150.0
2   150     150.0
3    暂无     150.0
4  1200    1200.0
5    暂无     150.0
6   460     460.0
7   460     460.0
8   600     600.0
9   150     150.0


In [26]:
#经纬度
def process_coordinates(df, coord_cols=['coord_x', 'coord_y'], region_col='板块'):
    for col in coord_cols:
        # 按板块求均值
        mean_by_region = df.groupby(region_col)[col].transform('mean')
        # 填充缺失值
        df[col] = df[col].fillna(mean_by_region)
        # 若板块全为空，再用全局均值
        df[col] = df[col].fillna(df[col].mean())
    return df

# 应用到训练集和测试集
train_price = process_coordinates(train_price)
test_price = process_coordinates(test_price)

# 检查结果
print(train_price[['板块', 'coord_x', 'coord_y']].head(10))
print(test_price[['板块', 'coord_x', 'coord_y']].head(10))

       板块     coord_x    coord_y
0   150.0  117.424278  40.975752
1   299.0  117.389228  41.091295
2   911.0  117.200934  40.747919
3  1102.0  117.767308  41.228803
4   295.0  117.334530  40.952530
5    78.0  117.542976  41.109076
6  1126.0  117.578940  40.759199
7  1126.0  117.578952  40.759211
8   878.0  117.347728  40.963461
9   547.0  117.505180  40.818017
       板块     coord_x    coord_y
0   367.0  117.389491  40.901030
1   606.0  117.376625  40.767478
2  1110.0  117.631276  41.063635
3   555.0  117.186216  41.163738
4   990.0  117.400114  40.959679
5   502.0  117.673512  41.322198
6   575.0  117.543679  40.961197
7   572.0  117.502731  41.017570
8   173.0  117.471311  41.122863
9   367.0  117.389229  40.898037


In [27]:
#城市、坐标及环线
def process_city_and_coord(df):
    df = df.copy()
    
    #坐标二次项
    if 'coord_x' in df.columns and 'coord_y' in df.columns:
        x = df['coord_x']
        y = df['coord_y']
        # 二次项
        df['coord_x2'] = x**2
        df['coord_y2'] = y**2
        df['coord_xy'] = x*y
    
    #环线处理
    if '环线' in df.columns and '城市' in df.columns:
        # 环线哑变量
        ring_dummies = pd.get_dummies(df['环线'], prefix='环线', drop_first=True)
        df = pd.concat([df, ring_dummies], axis=1)
    
    #城市哑变量
    if '城市' in df.columns:
        city_dummies = pd.get_dummies(df['城市'], prefix='城市', drop_first=True)
        df = pd.concat([df, city_dummies], axis=1)
    
    return df

# === 应用到训练集和测试集 ===
train_price = process_city_and_coord(train_price)
test_price = process_city_and_coord(test_price)

# === 检查处理结果 ===
cols_to_check = [c for c in train_price.columns if ('环线_' in c) or ('城市_' in c) or ('coord' in c)]
print(train_price[cols_to_check].head())

      coord_x    coord_y      coord_x2     coord_y2     coord_xy  环线_中环至外环  \
0  117.424278  40.975752  13788.461178  1679.012236  4811.548092     False   
1  117.389228  41.091295  13780.230830  1688.494557  4823.675441     False   
2  117.200934  40.747919  13736.058815  1660.392942  4775.694202     False   
3  117.767308  41.228803  13869.138859  1699.814233  4855.405198     False   
4  117.334530  40.952530  13767.391965  1677.109711  4805.145862     False   

   环线_二环内  环线_二至三环  环线_五至六环  环线_六环外  ...   城市_2   城市_3   城市_4   城市_5   城市_6  \
0   False     True    False   False  ...  False  False  False  False  False   
1   False    False     True   False  ...  False  False  False  False  False   
2   False    False     True   False  ...  False  False  False  False  False   
3   False    False    False    True  ...  False  False  False  False  False   
4   False    False    False   False  ...  False  False  False  False  False   

    城市_7   城市_8   城市_9  城市_10  城市_11  
0  False  False  

In [28]:
#再次查看数据
print("=====Train Price Data Info=====")
print(train_price.columns.tolist())

=====Train Price Data Info=====
['城市', '区域', '板块', '环线', 'Price', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室', '厅', '卫', '用途_别墅', '用途_商业办公类', '用途_商住两用', '用途_普通住宅', 'rel_floor', 'total_floor', 'est_floor', '得房率', '朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_东北', '朝向_西南', '朝向_西北', '结构_未知结构', '结构_混合结构', '结构_砖混结构', '结构_钢混结构', '装修_毛坯', '装修_简装', '装修_精装', '梯户比', '电梯', '交易年份', '首次交易', '交易间隔天数', '房屋年限_满两年', '房屋年限_满五年', '产权_共有', '周边_医院', '周边_公园', '周边_商业', '交通_地铁', '交通_公交', '平均建筑年代', '物 业 费_num', '结构_塔楼', '结构_板楼', '结构_塔板结合', '结构_平房', '产权_高价', '产权_中价', '产权_低价', '水电', '停车位_处理后', '容 积 率_处理后', '停车费用_处理后', 'coord_x2', 'coord_y2', 'coord_xy', '环线

In [29]:
#处理异常值

#数值特征列表
num_features = ['建筑面积', '房屋总数', '绿 化 率', '容 积 率_处理后', '物 业 费_num', '燃气费', '供热费',
    '停车位_处理后', 'coord_x', 'coord_y', '室', '厅', '卫',
    'rel_floor', 'total_floor', 'est_floor', '梯户比',
    '交易年份', '交易间隔天数', '平均建筑年代',
    '停车费用_处理后', 'coord_x2', 'coord_y2', 'coord_xy']

#哑变量列表
dummy_features = ['首次交易', '电梯', '水电', '用途_别墅', '用途_商业办公类', '用途_商住两用', '用途_普通住宅',
    '朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_东北', '朝向_西南', '朝向_西北',
    '结构_未知结构', '结构_混合结构', '结构_砖混结构', '结构_钢混结构',
    '结构_塔楼', '结构_板楼', '结构_塔板结合', '结构_平房',
    '装修_毛坯', '装修_简装', '装修_精装',
    '房屋年限_满两年', '房屋年限_满五年',
    '产权_共有', '产权_高价', '产权_中价', '产权_低价',
    '周边_医院', '周边_公园', '周边_商业',
    '交通_地铁', '交通_公交',
    '环线_中环至外环', '环线_二环内', '环线_二至三环', '环线_五至六环', '环线_六环外', '环线_内环内', '环线_内环至中环', '环线_内环至外环', '环线_四至五环', '环线_外环外',
    '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '城市_7', '城市_8', '城市_9', '城市_10', '城市_11']

others = ['区域', 'Price','ID']

#去除所有object列（除区域、价格）
keep_cols = num_features + dummy_features + others
train_price = train_price[train_price.columns.intersection(keep_cols)]
test_price = test_price[test_price.columns.intersection(keep_cols)]

def clean_numeric_outliers_by_region(df, numeric_cols, region_col='区域', method='iqr', factor=1.5):
    """
    按区域清理数值列异常值，并用区域中位数填充，若区域无有效值则用全局中位数。
    """
    df = df.copy()
    
    for col in numeric_cols:
        if method == 'iqr':
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - factor * IQR
            upper = Q3 + factor * IQR
        elif method == 'zscore':
            mean = df[col].mean()
            std = df[col].std()
            lower = mean - 3*std
            upper = mean + 3*std
        
        # 将异常值先设为 NaN
        df[col] = df[col].mask((df[col] < lower) | (df[col] > upper), np.nan)
        
        # 按区域填充中位数
        df[col] = df.groupby(region_col)[col].transform(
            lambda x: x.fillna(x.median())
        )
        
        # 对仍然缺失的值，用全局中位数填充
        global_median = df[col].median()
        df[col] = df[col].fillna(global_median)
    
    return df

train_price = clean_numeric_outliers_by_region(train_price, num_features, region_col='区域')
test_price = clean_numeric_outliers_by_region(test_price, num_features, region_col='区域')

train_price = train_price.drop(columns='区域')
test_price = test_price.drop(columns='区域')

#检查
# 把所有 bool 列转成 int
bool_cols = train_price.select_dtypes(include='bool').columns
train_price[bool_cols] = train_price[bool_cols].astype(int)
test_price[bool_cols] = test_price[bool_cols].astype(int)

# 找出 train 中不是 float 或 int 的列
non_numeric_train = train_price.select_dtypes(exclude=['float64', 'int64', 'int32']).columns
print("Train 非数值列:", list(non_numeric_train))

# 找出 test 中不是 float 或 int 的列
non_numeric_test = test_price.select_dtypes(exclude=['float64', 'int64', 'int32']).columns
print("Test 非数值列:", list(non_numeric_test))

Train 非数值列: []
Test 非数值列: []


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

scaler = StandardScaler()

# y 是目标变量（价格开根），X 是特征变量
y = np.log(train_price['Price'])
X = train_price.drop(columns=['Price'])

#划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=111
)

#标准化
X_train[num_features] = scaler.fit_transform(X_train[num_features])
X_test[num_features] = scaler.transform(X_test[num_features])

#加上常数项
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

#拟合OLS回归
model_OLS = sm.OLS(y_train, X_train).fit()

#查看回归结果
print(model_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     3278.
Date:                Wed, 29 Oct 2025   Prob (F-statistic):               0.00
Time:                        18:32:45   Log-Likelihood:                -42603.
No. Observations:               83096   AIC:                         8.537e+04
Df Residuals:                   83013   BIC:                         8.615e+04
Df Model:                          82                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          14.2193      0.076    188.120      

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

#验证
y_pred = model_OLS.predict(X_test)

from sklearn.metrics import r2_score

r2_val = r2_score(y_test, y_pred)
print("验证集 R²:", r2_val)

#交叉验证
model = LinearRegression()
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
print("交叉验证 R²:", np.mean(scores))

验证集 R²: 0.7611192967773792
交叉验证 R²: 0.7634532531415706


In [32]:
#Lasso
from sklearn.linear_model import Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score

# 使用交叉验证确定最优 alpha
lasso_cv = LassoCV(
    alphas=np.logspace(-4, 2, 50),
    cv=5,
    random_state=42,
    max_iter=10000
)
lasso_cv.fit(X_train, y_train)

best_alpha = lasso_cv.alpha_
print(f"✅ 最优 alpha: {best_alpha:.6f}")

# 用最优 alpha 重新训练 Lasso 模型
lasso_model = Lasso(alpha=best_alpha, max_iter=10000, random_state=42)
lasso_model.fit(X_train, y_train)

# R² in
y_pred_log_in = lasso_model.predict(X_train)
r2_log_in = r2_score(y_train, y_pred_log_in)
print(f"R²:in model: {r2_log_in:.6f}")

# 验证集预测 + R²（在 log 空间计算）
y_pred_log_out = lasso_model.predict(X_test)
r2_log_out = r2_score(y_test, y_pred_log_out)
print(f"R²:out of model: {r2_log_out:.6f}")

# 查看非零特征
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': lasso_model.coef_
})
important_features = coef_df[coef_df['Coefficient'] != 0].sort_values(by='Coefficient', ascending=False)

print("\n📊 非零特征数量:", len(important_features))
print(important_features.head(10))

#交叉验证
# 交叉验证的每折 MSE 路径：shape = (n_alphas, n_folds)
mse_path = lasso_cv.mse_path_  

# 平均 MSE
mean_mse = mse_path.mean(axis=1)

# 对应最优 alpha 的平均 MSE
best_alpha_idx = np.where(lasso_cv.alphas_ == lasso_cv.alpha_)[0][0]
best_cv_mse = mean_mse[best_alpha_idx]

# 对应训练集方差
y_var = np.var(y_train)

# 计算交叉验证 R² = 1 - MSE / Var(y)
best_cv_r2 = 1 - best_cv_mse / y_var
print(f"交叉验证 R² (alpha={best_alpha:.6f}): {best_cv_r2:.6f}")

✅ 最优 alpha: 0.000100
R²:in model: 0.763401
R²:out of model: 0.760222

📊 非零特征数量: 79
     Feature  Coefficient
11     用途_别墅     1.489714
78      城市_7     0.784251
63    环线_二环内     0.611356
14   用途_普通住宅     0.589917
61  coord_xy     0.386658
75      城市_4     0.367215
64   环线_二至三环     0.274305
67    环线_内环内     0.223034
68  环线_内环至中环     0.215742
69  环线_内环至外环     0.195926
交叉验证 R² (alpha=0.000100): 0.762838


In [36]:
# -------------------------------
#最佳线性
# -------------------------------
def best_linear_model(X_train, X_test, y_train, y_test):
    """
    OLS 最佳线性模型训练与验证
    """
    # -------------------------------
    # 模型训练（statsmodels OLS）
    # -------------------------------
    model = sm.OLS(y_train, X_train)
    results = model.fit()

    # -------------------------------
    # 输出模型信息（in-sample）
    # -------------------------------
    print("===== 模型拟合摘要（训练集） =====")
    print(results.summary())

    # in-sample R²
    r2_in = results.rsquared
    print(f"\n训练集（in-sample）R²: {r2_in:.4f}")

    # -------------------------------
    # 验证集预测
    # -------------------------------
    y_pred = results.predict(X_test)
    r2_out = r2_score(y_test, y_pred)
    mse_out = mean_squared_error(y_test, y_pred)
    print(f"\n验证集（out-of-sample）R²: {r2_out:.4f}")
    print(f"验证集 MSE: {mse_out:.4f}")

    # -------------------------------
    # 输出系数排序
    # -------------------------------
    coef = pd.Series(results.params, index=X_train.columns)
    coef_sorted = coef.sort_values(key=abs, ascending=False)
    print("\n前20个重要特征系数:")
    print(coef_sorted.head(20))

    # -------------------------------
    # sklearn LinearRegression 交叉验证
    # -------------------------------
    lr = LinearRegression()
    # 注意：sklearn 需要 numpy 数组或 DataFrame，不含 statsmodels 的常数列格式即可
    scores = cross_val_score(lr, X_train, y_train, cv=5, scoring='r2')
    print(f"\n交叉验证 R²: {np.mean(scores):.6f}")

    return results, coef_sorted, {'R2_in': r2_in, 'R2_out': r2_out, 'MSE_out': mse_out, 'CV_R2': np.mean(scores)}

# -------------------------------
# 调用函数
# -------------------------------
results, coef_sorted, metrics = best_linear_model(X_train, X_test, y_train, y_test)

===== 模型拟合摘要（训练集） =====
                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     3278.
Date:                Wed, 29 Oct 2025   Prob (F-statistic):               0.00
Time:                        18:42:55   Log-Likelihood:                -42603.
No. Observations:               83096   AIC:                         8.537e+04
Df Residuals:                   83013   BIC:                         8.615e+04
Df Model:                          82                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          14.2193    

In [38]:
# 对齐列，保证和训练集一致
feature_cols = X_train.columns.drop('const')  # 不含常数项
X_test_new = test_price.reindex(columns=feature_cols, fill_value=0)

# 标准化数值列
X_test_new[num_features] = scaler.transform(X_test_new[num_features])

# 添加常数项
X_test_new = sm.add_constant(X_test_new, has_constant='add')

# 预测
y_pred_log = results.predict(X_test_new)
y_pred = np.exp(y_pred_log)

# 输出
submission = pd.DataFrame({
    'ID': test_price['ID'],  # 用测试集原本的ID列
    'Price': y_pred
})
submission.to_csv('submission_price.csv', index=False)
print("已输出")

已输出
